# Training a Torch Classifier

This tutorial shows you how to train an image classifier using the [Ray AI Runtime](air) (AIR).

You should be familiar with [PyTorch](https://pytorch.org/) before starting the tutorial. If you need a refresher, read PyTorch's [training a classifier](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html) tutorial.

## Before you begin

* Install the [Ray AI Runtime](air). You'll need Ray 1.13 later to run this example.

In [1]:
!pip install 'ray[air]'

* Install `requests`, `torch`, and `torchvision`

In [2]:
!pip install requests torch torchvision

## Load and normalize CIFAR-10

We'll train our classifier on a popular image dataset called [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html).

First, let's load CIFAR-10 into a Ray Dataset.

In [3]:
import ray
from ray.data.datasource import SimpleTorchDatasource
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

def train_dataset_factory():
    return torchvision.datasets.CIFAR10(root="./data", download=True, train=True, transform=transform)

def test_dataset_factory():
    return torchvision.datasets.CIFAR10(root="./data", download=True, train=False, transform=transform)

train_dataset: ray.data.Dataset = ray.data.read_datasource(SimpleTorchDatasource(), dataset_factory=train_dataset_factory)
test_dataset: ray.data.Dataset = ray.data.read_datasource(SimpleTorchDatasource(), dataset_factory=test_dataset_factory)

2022-08-16 14:59:57,868	INFO worker.py:1481 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8266.
2022-08-16 14:59:58,968	WARNING read_api.py:291 -- ⚠️  The number of blocks in this dataset (1) limits its parallelism to 1 concurrent tasks. This is much less than the number of available CPU slots in the cluster. Use `.repartition(n)` to increase the number of dataset blocks.
(_get_read_tasks pid=9523) 2022-08-16 14:59:58,966	WARNING torch_datasource.py:55 -- `SimpleTorchDatasource` doesn't support parallel reads. The `parallelism` argument will be ignored.


(_execute_read_task pid=9523) Files already downloaded and verified


2022-08-16 15:00:04,407	WARNING read_api.py:291 -- ⚠️  The number of blocks in this dataset (1) limits its parallelism to 1 concurrent tasks. This is much less than the number of available CPU slots in the cluster. Use `.repartition(n)` to increase the number of dataset blocks.
(_get_read_tasks pid=9523) 2022-08-16 15:00:04,406	WARNING torch_datasource.py:55 -- `SimpleTorchDatasource` doesn't support parallel reads. The `parallelism` argument will be ignored.


(_execute_read_task pid=9523) Files already downloaded and verified


In [4]:
train_dataset

{py:class}`SimpleTorchDatasource <ray.data.datasource.SimpleTorchDatasource>` is unperformant, so you shouldn't use it with larger datasets.

Next, let's represent our data using pandas dataframes instead of tuples. This lets us call {py:meth}`Dataset.iter_torch_batches <ray.data.Dataset.iter_torch_batches>` later in the tutorial.

In [5]:
from typing import Tuple
import pandas as pd
import torch


def convert_batch_to_pandas(batch: Tuple[torch.Tensor, int]) -> pd.DataFrame:
    images = [image.numpy() for image, _ in batch]
    labels = [label for _, label in batch]
    return pd.DataFrame({"image": images, "label": labels})


train_dataset = train_dataset.map_batches(convert_batch_to_pandas)
test_dataset = test_dataset.map_batches(convert_batch_to_pandas)

Read->Map_Batches:   0%|          | 0/1 [00:00<?, ?it/s]

(_map_block_nosplit pid=9523) Files already downloaded and verified


Read->Map_Batches:   0%|          | 0/1 [00:00<?, ?it/s]

(_map_block_nosplit pid=9523) Files already downloaded and verified


Read->Map_Batches: 100%|██████████| 1/1 [00:01<00:00,  1.38s/it]


In [6]:
train_dataset

## Train a convolutional neural network

Now that we've created our datasets, let's define the training logic.

We define our training logic in a function called `train_loop_per_worker`. This function contains regular PyTorch code with a few notable exceptions:
* We wrap our model with {py:func}`train.torch.prepare_model <ray.train.torch.prepare_model>`.
* We call {py:func}`session.get_dataset_shard <ray.air.session.get_dataset_shard>` and {py:meth}`Dataset.iter_torch_batches <ray.data.Dataset.iter_torch_batches>` to get a subset of our training data.
* We save model state using {py:func}`session.report <ray.air.session.report>`.

In [7]:
from ray import train
from ray.air import session, Checkpoint
from ray.train.torch import TorchCheckpoint
import torch.nn as nn
import torch.optim as optim
import torchvision


def train_loop_per_worker(config):
    model = torchvision.models.resnet18()
    model = train.torch.prepare_model(model)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    train_dataset_shard = session.get_dataset_shard("train")

    for epoch in range(1):
        running_loss = 0.0
        train_dataset_batches = train_dataset_shard.iter_torch_batches(
            batch_size=config["batch_size"],
        )
        for i, batch in enumerate(train_dataset_batches):
            # get the inputs and labels
            inputs, labels = batch["image"], batch["label"]

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:  # print every 2000 mini-batches
                print(f"[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}")
                running_loss = 0.0

        metrics = dict(running_loss=running_loss)
        checkpoint = TorchCheckpoint.from_model(model=model.module)
        session.report(metrics, checkpoint=checkpoint)

Finally, we can train our model. This should take a few minutes to run.

In [8]:
from ray.train.torch import TorchTrainer
from ray.air.config import ScalingConfig

trainer = TorchTrainer(
    train_loop_per_worker=train_loop_per_worker,
    train_loop_config={"batch_size": 2},
    datasets={"train": train_dataset},
    scaling_config=ScalingConfig(num_workers=2)
)
result = trainer.fit()
latest_checkpoint = result.checkpoint

Trial name,status,loc
TorchTrainer_ccca4_00000,RUNNING,127.0.0.1:9529


(RayTrainWorker pid=9531) 2022-08-16 15:00:14,135	INFO config.py:71 -- Setting up process group for: env:// [rank=0, world_size=2]
(RayTrainWorker pid=9531) 2022-08-16 15:00:15,637	INFO train_loop_utils.py:300 -- Moving model to device: cpu
(RayTrainWorker pid=9531) 2022-08-16 15:00:15,638	INFO train_loop_utils.py:347 -- Wrapping provided model in DDP.
(RayTrainWorker pid=9531) /Users/bveeramani/GitHub/ray/python/ray/air/_internal/torch_utils.py:122: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:178.)
(RayTrainWorker pid=9531)   return torch.as_tensor(ndarray, dtype=dtype, device=device)
(

(RayTrainWorker pid=9531) [1,  2000] loss: 2.746
(RayTrainWorker pid=9532) [1,  2000] loss: 2.763
(RayTrainWorker pid=9531) [1,  4000] loss: 2.654
(RayTrainWorker pid=9532) [1,  4000] loss: 2.645
(RayTrainWorker pid=9531) [1,  6000] loss: 2.633
(RayTrainWorker pid=9532) [1,  6000] loss: 2.621
(RayTrainWorker pid=9531) [1,  8000] loss: 2.613
(RayTrainWorker pid=9532) [1,  8000] loss: 2.623


To scale your training script, create a [Ray Cluster](cluster-index) and increase the number of workers. If your cluster contains GPUs, add `"use_gpu": True` to your scaling config.

```{code-block} python
scaling_config=ScalingConfig(num_workers=8, "use_gpu=True)
```

## Test the network on the test data

Let's see how our model performs.

To classify images in the test dataset, we'll need to create a {py:class}`Predictor <ray.train.predictor.Predictor>`.

{py:class}`Predictors <ray.train.predictor.Predictor>` load data from checkpoints and efficiently perform inference. In contrast to {py:class}`TorchPredictor <ray.train.torch.TorchPredictor>`, which performs inference on a single batch, {py:class}`BatchPredictor <ray.train.batch_predictor.BatchPredictor>` performs inference on an entire dataset. Because we want to classify all of the images in the test dataset, we'll use a {py:class}`BatchPredictor <ray.train.batch_predictor.BatchPredictor>`.

In [ ]:
from ray.train.torch import TorchPredictor
from ray.train.batch_predictor import BatchPredictor

batch_predictor = BatchPredictor.from_checkpoint(
    checkpoint=latest_checkpoint,
    predictor_cls=TorchPredictor,
    model=torchvision.models.resnet18(),
)

outputs: ray.data.Dataset = batch_predictor.predict(
    data=test_dataset, dtype=torch.float, feature_columns=["image"], keep_columns=["label"]
)

Our model outputs a list of energies for each class. To classify an image, we
choose the class that has the highest energy.

In [ ]:
import numpy as np

def convert_logits_to_classes(df):
    best_class = df["predictions"].map(lambda x: x.argmax())
    df["prediction"] = best_class
    return df[["prediction", "label"]]

predictions = outputs.map_batches(
    convert_logits_to_classes
)

predictions.show(1)

Now that we've classified all of the images, let's figure out which images were
classified correctly. The ``predictions`` dataset contains predicted labels and 
the ``test_dataset`` contains the true labels. To determine whether an image 
was classified correctly, we join the two datasets and check if the predicted 
labels are the same as the actual labels.

In [ ]:
def calculate_prediction_scores(df):
    df["correct"] = df["prediction"] == df["label"]
    return df

scores = predictions.map_batches(calculate_prediction_scores)

scores.show(1)

To compute our test accuracy, we'll count how many images the model classified 
correctly and divide that number by the total number of test images.

In [ ]:
scores.sum(on="correct") / scores.count()

## Deploy the network and make a prediction

Our model seems to perform decently, so let's deploy the model to an 
endpoint. This'll allow us to make predictions over the Internet.

In [ ]:
from ray import serve
from ray.serve import PredictorDeployment
from ray.serve.http_adapters import json_to_numpy


serve.run(
    PredictorDeployment.bind(
        TorchPredictor,
        latest_checkpoint,
        model=torchvision.models.resnet18(),
        http_adapter=json_to_numpy,
    )
)

Let's classify a test image.

In [ ]:
image = test_dataset.take(1)[0]["image"]

In [ ]:
array.shape

You can perform inference against a deployed model by posting a dictionary with an `"array"` key. To learn more about the default input schema, read the {py:class}`NdArray <ray.serve.http_adapters.NdArray>` documentation.

In [ ]:
import requests

payload = {"array": image.tolist(), "dtype": "float32"}
response = requests.post("http://localhost:8000/", json=payload)
response.json()